# Brute-force

In [ ]:
import numpy as np

## Experimental settings

In [9]:
MAX_LENGTH = 15
TARGET_UNITARY = np.array([[0, -1],
                           [1,  0]], dtype=complex)

## Fitness function

In [ ]:
allowed_indices = [0, 1, 2, 3]
inverse_map = {0: 2, 1: 3, 2: 0, 3: 1}

phi = (1 + np.sqrt(5)) / 2

sigma_1 = np.array([
    [np.exp(-4j * np.pi / 5), 0],
    [0, np.exp(3j * np.pi / 5)]
])
sigma_2 = np.array([
    [np.exp(4j * np.pi / 5) / phi,
     np.exp(-3j * np.pi / 5) / np.sqrt(phi)],
    [np.exp(-3j * np.pi / 5) / np.sqrt(phi), -1 / phi]
])
sigma_1_inv = np.linalg.inv(sigma_1)
sigma_2_inv = np.linalg.inv(sigma_2)

generators_map = {
    0: sigma_1,
    1: sigma_2,
    2: sigma_1_inv,
    3: sigma_2_inv
}

def fitness(seq, unitary):
    """
    seq의 각 원소는 0,1,2,3 중 하나로, 내부적으로 매핑된 행렬들을 차례대로 곱합니다.
    최종적으로 얻은 행렬과 phase-adjusted identity 사이의 Frobenius norm 거리를 반환합니다.
    """
    M = np.array(unitary, copy=True)
    for idx in seq:
        gate = generators_map[idx]
        M = np.dot(M, gate)
    trace_M = np.trace(M)
    if abs(trace_M) > 1e-12:
        phase = trace_M / abs(trace_M)
    else:
        phase = 1.0
    optimal_matrix = phase * np.eye(2, dtype=complex)
    distance = np.linalg.norm(M - optimal_matrix, 'fro')
    return distance

## Brute-force search

In [11]:
def generate_sequences(current_seq, length):
    """
    current_seq를 시작으로 하여, 총 길이 length인 시퀀스를 재귀적으로 생성합니다.
    단, 인접한 두 원소가 서로의 역원이 되는 경우(예: 0 다음에 2, 2 다음에 0, 1 다음에 3, 3 다음에 1)는 배제합니다.
    """
    if len(current_seq) == length:
        yield current_seq
    else:
        for x in allowed_indices:
            # 만약 current_seq가 비어있거나, 마지막 원소와 x가 역원 관계가 아니라면
            if not current_seq or x != inverse_map[current_seq[-1]]:
                yield from generate_sequences(current_seq + [x], length)

# --- brute-force 탐색 ---
best_seq = None
best_fit = float('inf')
total_count = 0

print("Brute-force 탐색 시작...")
for seq in generate_sequences([], MAX_LENGTH):
    total_count += 1
    f_val = fitness(seq, TARGET_UNITARY)
    if f_val < best_fit:
        best_fit = f_val
        best_seq = seq
        print(f"새로운 best 발견: {best_seq} with fitness = {best_fit}")
        
print("\n총 생성된 시퀀스 개수:", total_count)
print("최종 best sequence:", best_seq)
print("최종 best fitness:", best_fit)


Brute-force 탐색 시작...
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] with fitness = 2.0000000000000004
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] with fitness = 1.4668494320614869
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] with fitness = 1.0046404518819458
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0] with fitness = 1.0046404518819456
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0] with fitness = 1.0046404518819438
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 0, 1] with fitness = 1.0046404518819436
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0] with fitness = 0.5507218106069718
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 3, 0] with fitness = 0.5507218106069716
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 3, 3] with fitness = 0.5507218106069702
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 0, 3] with fitness = 0.55072181060697
새로운 best 발견: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0